![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

### `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

### `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

### `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

* Subset, clean, and reformat the bank_marketing.csv dataset to create and store three new files based on the requirements detailed in the notebook.

* Split and tidy bank_marketing.csv, storing as three DataFrames called client, campaign, and economics, each containing the columns outlined in the notebook and formatted to the data types listed.

* Save the three DataFrames to csv files, without an index, as client.csv, campaign.csv, and economics.csv respectively.

## Solution

In [7]:
import pandas as pd
import numpy as np

In [2]:
# loading the data into pandas dataframe
data = pd.read_csv('bank_marketing.csv')

# let's check the data
data

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional.course,no,yes,nov,30,334,1,0,nonexistent,94.767,1.028,yes
41184,41184,46,blue-collar,married,professional.course,no,no,nov,6,383,1,0,nonexistent,94.767,1.028,no
41185,41185,56,retired,married,university.degree,no,yes,nov,24,189,2,0,nonexistent,94.767,1.028,no
41186,41186,44,technician,married,professional.course,no,no,nov,17,442,1,0,nonexistent,94.767,1.028,yes


In [22]:
pd.options.mode.copy_on_write = True
data = pd.read_csv("bank_marketing.csv")
print(data["day"])

# let's the dataset into the dataframes
client_df = data[
    ["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]
]

campaign_df = data[
    [
        "client_id",
        "number_contacts",
        "day",
        "month",
        "contact_duration",
        "previous_campaign_contacts",
        "previous_outcome",
        "campaign_outcome",
    ]
]

economics_df = data[["client_id", "cons_price_idx", "euribor_three_months"]]


# let's begin cleaning the client dataframe
client_df["job"] = client_df["job"].str.replace(".", "_")
client_df["education"] = client_df["education"].str.replace(".", "_")
client_df["education"] = client_df["education"].replace("unknown", np.NaN)
client_df["credit_default"] = client_df["credit_default"].map(
    {"yes": 1, "no": 0, "unknown": 0}
)
client_df["credit_default"] = client_df["credit_default"].astype(bool)
client_df["mortgage"] = client_df["mortgage"].map({"yes": 1, "no": 0, "unknown": 0})
client_df["mortgage"] = client_df["mortgage"].astype(bool)

# let's begin cleaning the campaign dataframe
campaign_df["previous_outcome"] = campaign_df["previous_outcome"].map(
    {"success": 1, "failure": 0, "nonexistent": 0}
)
campaign_df["previous_outcome"] = campaign_df["previous_outcome"].astype(bool)
campaign_df["campaign_outcome"] = campaign_df["campaign_outcome"].map(
    {"yes": 1, "no": 0}
)
campaign_df["campaign_outcome"] = campaign_df["campaign_outcome"].astype(bool)

# let's add the 'year' column to the campaign dataframe
campaign_df["year"] = "2022"
campaign_df["day"] = campaign_df["day"].astype(str)

campaign_df["last_contact_date"] = (
    campaign_df["year"] + "-" + campaign_df["month"] + "-" + campaign_df["day"]
)
campaign_df["last_contact_date"] = pd.to_datetime(
    campaign_df["last_contact_date"], format="%Y-%b-%d"
)

# let's remove the day, month and year column
campaign_df.drop(columns=["day", "month", "year"], inplace=True)

# let's save the dataframes as .csv file
client_df.to_csv("client.csv", index=False)
campaign_df.to_csv("campaign.csv", index=False)
economics_df.to_csv("economics.csv", index=False)

0        13
1        19
2        23
3        27
4         3
         ..
41183    30
41184     6
41185    24
41186    17
41187    23
Name: day, Length: 41188, dtype: int64
